Reproducing Deep Image Prior paper
=====================

In this notebook we reproduce Table 1 in Deep Image Prior paper. Although the authors of the paper made the code available online, this is out approach to independently reproduce thair results. 
We developed the code in Jupyter notebook and it is compatible with Google Colab platform to be used with GPU.


## What is Deep Image Prior

In the paper, the authors argue that a great
deal of image statistics are captured by the structure of
a convolutional image generator independent of learning.
What it means is that we can train the generator netowrk on a single degraded image, instead of large dataset of example images,
to reconstruct the image. In this scheme, the network weights serve
as a parametrization of the restored image.


## How does it work?

Lets assume that our image x is under following process:

$ x \Rightarrow Degradation \Rightarrow  \hat{x} \Rightarrow  Restoration \Rightarrow x^{*} $

Our goal is to find $ x^{*}$.
We can do that by finding the MAP estimate of our posterior distribution of clean images:

\begin{equation}
MAP: x^{*} = argmax_{x} p(x|\hat{x})
\end{equation}

As it is usually the case, obtaining posterior distribution $p(x|\hat{x})$ is intractable. We can rewrite the equation using Bayes theorem:

\begin{equation}
p(x|\hat{x}) = \frac{p(\hat{x}|x)p(x)}{p(\hat{x})} \sim p(\hat{x}|x)p(x)
\end{equation}


to be continued...
